# Genetic Algorithm for string of 1

In [1]:
import numpy as np
import random

In [2]:
def createPopulation():
    population = [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]
    #random.seed(1)     #Prevent from randomly generated
    for i in range(maxPop):
        for j in range(maxS):
            population[i][j] = random.randint(0,1)
    return population

In [3]:
def createChromosome(population):
    chromosome = []
    for i in range(maxPop):
        pop_str = ''.join(map(str, population[i]))
        chromosome.insert(i, pop_str)
    return chromosome

In [4]:
def printPopulation(population):
    print('Initial Population:')
    for i in range(maxPop):
        print(population[i], sep=' ')
    print()

In [5]:
def printFitness(fitness):
    print("Fitness: ")
    for i in range(maxPop):
        print(fitness[i], sep = ' ')
    #print()

In [6]:
def printChromosomeFitness(chromfit):
    print("Chromosome fitness:")
    for i in range(maxPop):
        (chrom, fit) = extractChromosomeFitness(chromfit[i])
        print("{} Fitness: {}".format(chrom, fit))
    #print()

In [7]:
def calculateFitness(chromosome):
    total = 0
    tot_chromosome = [0,0,0,0]
    fitness = [0.0,0.0,0.0,0.0]
    
    for i in range(maxPop):
        tot_chromosome[i] = int(chromosome[i], 2)**2
        total = total + tot_chromosome[i]

    for i in range(maxPop):
        fitness[i] = round((tot_chromosome[i] / total)*100, 2)
    
    return (fitness)

In [8]:
def extractChromosomeFitness(chromf):
    chrs = chromf[0:5]
    fits = chromf[6:12]
    return (chrs, fits)

In [9]:
def mergePopulationFitness(chromosome, fitness):
    chromfit = ["","","",""]
    strfit = ""
    i=0
    for r in chromosome:
        strfit = r + ","+''.join(str(fitness[i]))
        chromfit[i] = strfit
        i = i + 1 
        strfit=""
    return (chromfit)

In [10]:
def selectChromosomes(chromfit):
    newchromosomes = []
    ct = 0
    fittest = 0.0
    newchrom = ""
    for i in range(maxPop):
        (chrom, fit) = extractChromosomeFitness(chromfit[i])
        if float(fit) > 10.0:
            if float(fit) > fittest:
                fittest = float(fit)
                newchrom = chrom
            newchromosomes.insert(ct, chrom)
            ct = ct + 1
    
    for i in range(maxPop-ct):
        newchromosomes.insert(i, newchrom)

    return (newchromosomes)

In [11]:
def getOnePointCrossOver():
    cr = np.array([-9,-9], dtype = int)
    cr[0] = random.randint(0,4)
    cr[1] = random.randint(0,4)
    return(cr)

In [12]:
def pickMate():
    mate_order = np.array([-9,-9,-9,-9], dtype = int)
    ct = 0
    while ct < 4:
        m = random.randint(0,3)
        if m in set(mate_order):
            m = random.randint(0,3)
        else:
            mate_order[ct] = m
            ct = ct + 1 
    return (mate_order)

In [13]:
def crossOver(cr, mate, chromosome):
    k=0
    newchromosomes = []
    numOfMate = int(maxPop/2)
    for j in range(numOfMate):
        p1 = mate[k]
        p2 = mate[k+1]
        crp = cr[j]
        parent1 = chromosome[p1]
        parent2 = chromosome[p2]
        substr1 = parent1[crp+1:maxS]
        substr2 = parent2[crp+1:maxS]
        newparent1 = parent1[0:crp+1] + substr2
        newparent2 = parent2[0:crp+1] + substr1
        newchromosomes.insert(k, newparent1)
        newchromosomes.insert(k+1, newparent2)
        k = k + 2
    
    return(newchromosomes)

In [14]:
def mutation(chromosome):
    mutated = []
    selected = []
    pm = int((maxPop * maxS)*0.1)  #Calculate probability of mutation
    for i in range(pm):
        mut = random.randint(0,4)  # choose which gen to mutate
        sel = random.randint(0,3)  # choose which chromosome to mutate
        selected = chromosome[sel]
        #print("Selected Chromosome {} before mutated at {}: ".format(sel+1, mut+1))
        #print(selected)
        newsel = list(selected)
        if selected[mut] == '0':
            newsel[mut] = '1'
        else:
            newsel[mut] = '0'
        selected = "".join(newsel)
        chromosome[sel] = selected
        #print("New Chromosome {} after mutated at {}: ".format(sel+1, mut+1))
        #print(chromosome)
    
    return (chromosome)


In [15]:
def calculateObjective(chromosome):
    total = 0
    tot_chromosome = [0,0,0,0]
    
    for i in range(maxPop):
        tot_chromosome[i] = int(chromosome[i], 2)**2
        total = total + tot_chromosome[i]
    return (total)    

In [16]:
def allOnes(n):
    return ((n+1) & n == 0) and (n!=0)

def optimalObjctive():
    bins = bin(2**maxS-1)
    numstr = str(bins[2:])
    optObj = 4*(int(numstr,2)**2)
    return(optObj)

In [17]:
maxPop = 4
maxS = 5

def main():
    print('"Genetic Algorithm"')
    population = createPopulation()
    chromosome = createChromosome(population)
    print("Initial chomosome: {}".format(chromosome))
    totObj = calculateObjective(chromosome)
    optSol = optimalObjctive()
    gen = 0
    while totObj < optSol:
        fitness = calculateFitness(chromosome)
        chromFitness = mergePopulationFitness(chromosome, fitness)
        #printChromosomeFitness(chromFitness)
        totObj = calculateObjective(chromosome)
        if totObj < optSol:
            #print("Objective function value = {} of {}".format(totObj, optSol))
            chromosome = selectChromosomes(chromFitness)
            cr = getOnePointCrossOver()
            mate = pickMate()
            chromosome = crossOver(cr, mate, chromosome)
            chromosome = mutation(chromosome)
            #print("\nNew chomosome: {}".format(chromosome))
            gen = gen + 1
    print("Final chromosome: {}".format(chromosome))
    print("Optimal Solution found in {} generations".format(gen))
    print("Optimal objective value = {}".format(totObj))
        
if __name__ == "__main__":
    main()

"Genetic Algorithm"
Initial chomosome: ['11000', '01101', '10001', '01101']
Final chromosome: ['11111', '11111', '11111', '11111']
Optimal Solution found in 776 generations
Optimal objective value = 3844
